In [7]:
import numpy as np
import pathlib
import os

In [23]:
input_path = os.path.join(os.path.join(pathlib.Path(globals()['_dh'][0]).parent, "data"), "js-fakes-16thSeparated.npz")
jsf = np.load(input_path, allow_pickle=True, encoding='latin1')